In [1]:
#Import dependencies
import pandas as pd

In [4]:
#Read in h1n1 data
swine_data = pd.read_csv('resources/h1n1.csv', encoding='latin_1')
#Check data is read in
swine_data.head()

,Country,Cases,Deaths,Update Time
0,Algeria,5,0.0,7/6/2009 9:00
1,Antigua and Barbuda,2,0.0,7/6/2009 9:00
2,Argentina,2485,60.0,7/6/2009 9:00
3,Australia,5298,10.0,7/6/2009 9:00
4,Austria,19,0.0,7/6/2009 9:00


In [11]:
#Clean country names
countries = swine_data.Country.to_list()
#Remove problematic characters from the names
country_clean = []
for country in countries:
    country_clean.append(country.strip('\xa0\xa0\xa0\xa0\xa0'))
#Replace Country column
swine_data['Country'] = country_clean
swine_data.head()

,Country,Cases,Deaths,Update Time
0,Algeria,5,0.0,7/6/2009 9:00
1,Antigua and Barbuda,2,0.0,7/6/2009 9:00
2,Argentina,2485,60.0,7/6/2009 9:00
3,Australia,5298,10.0,7/6/2009 9:00
4,Austria,19,0.0,7/6/2009 9:00


In [25]:
#Remove Grand Total
swine_data.drop(1821,axis=0, inplace=True)
swine_data.tail()

,Country,Cases,Deaths,Update Time
1816,Switzerland,2,0.0,5/23/2009 8:00
1817,Thailand,2,0.0,5/23/2009 8:00
1818,Turkey,2,0.0,5/23/2009 8:00
1819,United Kingdom,117,0.0,5/23/2009 8:00
1820,United States of America,6552,9.0,5/23/2009 8:00


In [32]:
#Set data to the week progression in pandemic
#Assign dates to week in the pandemic
times = swine_data['Update Time'].to_list()
weeks = []
for time in times:
    if time == '5/23/2009 8:00' or time == '5/25/2009 8:00' or time == '5/26/2009 6:00' or time == '5/27/2009 8:00' or time == '5/29/2009 6:00':
        weeks.append(1)
    elif time == '6/8/2009 6:00' or time == '6/5/2009 6:00' or time == '6/3/2009 6:00' or time == '6/1/2009 6:00':
        weeks.append(2)
    elif time == '6/17/2009 12:00' or time == '6/15/2009 17:00' or time == '6/12/2009 7:00' or time == '6/11/2009 14:00' or time == '6/10/2009 6:00':
        weeks.append(3)
    elif time == '6/26/2009 7:00' or time == '6/24/2009 7:00' or time == '6/22/2009 7:00' or time == '6/19/2009 7:00':
        weeks.append(4)
    elif time == '7/6/2009 9:00' or time == '7/3/2009 9:00' or time == '7/1/2009 9:00' or time == '6/29/2009 9:00':
        weeks.append(5)
#Add week number to df
swine_data['Week'] = weeks
#Drop Update Time
swine_data.drop('Update Time', axis=1, inplace=True)
swine_data.head()

,Country,Cases,Deaths,Week
0,Algeria,5,0.0,5
1,Antigua and Barbuda,2,0.0,5
2,Argentina,2485,60.0,5
3,Australia,5298,10.0,5
4,Austria,19,0.0,5
